In [1]:
import pandas as pd

In [3]:
 df = pd.DataFrame({
        'Transactions': ['T1','T1','T1','T2','T3','T3','T3','T3','T4','T4','T4','T5','T5','T6','T6','T6','T6','T7','T7','T8','T8','T9','T9','T9','T9','T10','T10'], 
        'Item': ['bread','butter','milk','milk','bread','milk','sugar','tea','bread','butter','milk','milk','cereals','milk','coffee','sugar','tea','milk','bread','cereals','butter','bread','cereals','sugar','tea','bread','coffee']})
df.head()

,Transactions,Item
0,T1,bread
1,T1,butter
2,T1,milk
3,T2,milk
4,T3,bread


In [6]:
basket = df.groupby(['Transactions'])['Item']
print(basket)